In [1]:
# %matplotlib ipympl
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *

In [4]:
dgilib_path = "C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Python\\dgilib.dll"

In [5]:
fig = plt.figure(figsize=(8, 6))
fig.show()

In [54]:
data = []

config_dict = {
    "power_buffers": [{"channel": CHANNEL_A, "power_type": POWER_CURRENT}],
    "read_mode": [True, True, True, True],
    "write_mode": [False, False, False, False],
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, True],
    "gpio_delay_time" : 0.010795,
    "fig": fig,
    "verbose": 0,
}

fig.clf()

with DGILibExtra(dgilib_path, **config_dict) as dgilib:
    dgilib.device_reset()
    data = dgilib.logger(1)
#     dgilib.ax.plot(*dgilib.power_filter_by_pin(2))
#     dgilib.ax.plot(*dgilib.power_filter_by_pin(3))

In [47]:
data[INTERFACE_GPIO]

([0.0015232,
  0.012914133333333334,
  0.02299786666666667,
  0.0330784,
  0.0431584,
  0.0532384,
  0.06331893333333334,
  0.0733984,
  0.08347840000000001,
  0.09355786666666667,
  0.10363733333333333,
  0.11371573333333333,
  0.12379573333333334,
  0.13387573333333333,
  0.1439552,
  0.15403466666666668,
  0.16411413333333333,
  0.17419306666666667,
  0.18427253333333335,
  0.194352,
  0.204432,
  0.21451093333333335,
  0.22459093333333333,
  0.2346704,
  0.24474986666666668,
  0.2548282666666667,
  0.2649072,
  0.2749856,
  0.28506560000000003,
  0.2951450666666667,
  0.3052256,
  0.31530400000000003,
  0.325384,
  0.33546346666666665,
  0.345544,
  0.3556224,
  0.36570186666666665,
  0.3757802666666667,
  0.3858592,
  0.39593706666666667,
  0.40601706666666665,
  0.4160954666666667,
  0.4261744,
  0.4362528,
  0.44633280000000003,
  0.4564128,
  0.4664917333333333,
  0.4765706666666667,
  0.48665013333333335,
  0.496728,
  0.5068074666666667,
  0.5168858666666667,
  0.5269648,
  0